In [2]:
import sqlite3
import os
from datetime import datetime

DB_FILE = 'loja.db'

if os.path.exists(DB_FILE):
    os.remove(DB_FILE)

conn = None
cursor = None

try:
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    print(f"Conexão com o banco de dados '{DB_FILE}' estabelecida.")
except sqlite3.Error as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
    exit()

if conn:
    try:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Clientes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                nome TEXT NOT NULL,
                email TEXT UNIQUE NOT NULL
            )
        ''')
        print("Tabela 'Clientes' criada ou já existente.")

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Produtos (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                nome_produto TEXT NOT NULL,
                preco REAL NOT NULL
            )
        ''')
        print("Tabela 'Produtos' criada ou já existente.")

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Vendas (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                id_cliente INTEGER NOT NULL,
                id_produto INTEGER NOT NULL,
                quantidade INTEGER NOT NULL,
                data_venda TEXT NOT NULL,
                FOREIGN KEY (id_cliente) REFERENCES Clientes(id),
                FOREIGN KEY (id_produto) REFERENCES Produtos(id)
            )
        ''')
        print("Tabela 'Vendas' criada ou já existente.")

        conn.commit()
    except sqlite3.Error as e:
        print(f"Erro ao criar tabelas: {e}")
        conn.rollback()

    print("\n--- Inserindo dados ---")

    clientes_data = [
        ('Ana Silva', 'ana.silva@email.com'),
        ('Bruno Costa', 'bruno.costa@email.com'),
        ('Carla Dias', 'carla.dias@email.com')
    ]
    try:
        cursor.executemany("INSERT INTO Clientes (nome, email) VALUES (?, ?)", clientes_data)
        conn.commit()
        print("Clientes inseridos.")
    except sqlite3.Error as e:
        print(f"Erro ao inserir clientes: {e}")
        conn.rollback()

    produtos_data = [
        ('Notebook Xpto', 3500.00),
        ('Mouse Gamer', 150.00),
        ('Teclado Mecânico', 300.00),
        ('Monitor UltraWide', 1800.00)
    ]
    try:
        cursor.executemany("INSERT INTO Produtos (nome_produto, preco) VALUES (?, ?)", produtos_data)
        conn.commit()
        print("Produtos inseridos.")
    except sqlite3.Error as e:
        print(f"Erro ao inserir produtos: {e}")
        conn.rollback()

    vendas_data = [
        (1, 1, 1, datetime.now().strftime('%Y-%m-%d %H:%M:%S')),
        (2, 2, 2, datetime.now().strftime('%Y-%m-%d %H:%M:%S')),
        (1, 3, 1, datetime.now().strftime('%Y-%m-%d %H:%M:%S')),
        (3, 4, 1, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    ]
    try:
        cursor.executemany("INSERT INTO Vendas (id_cliente, id_produto, quantidade, data_venda) VALUES (?, ?, ?, ?)", vendas_data)
        conn.commit()
        print("Vendas inseridas.")
    except sqlite3.Error as e:
        print(f"Erro ao inserir vendas: {e}")
        conn.rollback()

    print("\n--- Consultas (SELECT) ---")

    print("\nTodos os Clientes:")
    try:
        cursor.execute("SELECT id, nome, email FROM Clientes")
        clientes = cursor.fetchall()
        for cliente in clientes:
            print(cliente)
    except sqlite3.Error as e:
        print(f"Erro na consulta de clientes: {e}")

    print("\nProdutos com preço > R$200.00:")
    try:
        cursor.execute("SELECT nome_produto, preco FROM Produtos WHERE preco > ?", (200.00,))
        produtos_caros = cursor.fetchall()
        for produto in produtos_caros:
            print(produto)
    except sqlite3.Error as e:
        print(f"Erro na consulta de produtos caros: {e}")

    print("\nDetalhes das Vendas (com Nomes de Cliente e Produto):")
    try:
        cursor.execute('''
            SELECT
                V.id AS VendaID,
                C.nome AS NomeCliente,
                P.nome_produto AS NomeProduto,
                V.quantidade,
                P.preco AS PrecoUnitario,
                (V.quantidade * P.preco) AS ValorTotal,
                V.data_venda
            FROM Vendas AS V
            JOIN Clientes AS C ON V.id_cliente = C.id
            JOIN Produtos AS P ON V.id_produto = P.id
            ORDER BY V.data_venda DESC
        ''')
        vendas_detalhes = cursor.fetchall()
        for venda in vendas_detalhes:
            print(f"ID Venda: {venda[0]}, Cliente: {venda[1]}, Produto: {venda[2]}, Quantidade: {venda[3]}, Preço Unitário: {venda[4]}, Total: {venda[5]:.2f}, Data: {venda[6]}")
    except sqlite3.Error as e:
        print(f"Erro na consulta de vendas detalhadas: {e}")

    print("\nProdutos comprados por Ana Silva:")
    try:
        cursor.execute('''
            SELECT DISTINCT
                P.nome_produto
            FROM Produtos AS P
            JOIN Vendas AS V ON P.id = V.id_produto
            JOIN Clientes AS C ON V.id_cliente = C.id
            WHERE C.nome = 'Ana Silva'
        ''')
        produtos_ana = cursor.fetchall()
        for produto in produtos_ana:
            print(f"- {produto[0]}")
    except sqlite3.Error as e:
        print(f"Erro na consulta de produtos da Ana: {e}")

    print("\nTotal gasto por Cliente:")
    try:
        cursor.execute('''
            SELECT
                C.nome AS NomeCliente,
                SUM(V.quantidade * P.preco) AS GastoTotal
            FROM Vendas AS V
            JOIN Clientes AS C ON V.id_cliente = C.id
            JOIN Produtos AS P ON V.id_produto = P.id
            GROUP BY C.nome
            ORDER BY GastoTotal DESC
        ''')
        gastos_clientes = cursor.fetchall()
        for cliente in gastos_clientes:
            print(f"Cliente: {cliente[0]}, Gasto Total: R${cliente[1]:.2f}")
    except sqlite3.Error as e:
        print(f"Erro na consulta de gasto por cliente: {e}")

    if conn:
        conn.close()
        print(f"\nConexão com o banco de dados '{DB_FILE}' fechada.")

Conexão com o banco de dados 'loja.db' estabelecida.
Tabela 'Clientes' criada ou já existente.
Tabela 'Produtos' criada ou já existente.
Tabela 'Vendas' criada ou já existente.

--- Inserindo dados ---
Clientes inseridos.
Produtos inseridos.
Vendas inseridas.

--- Consultas (SELECT) ---

Todos os Clientes:
(1, 'Ana Silva', 'ana.silva@email.com')
(2, 'Bruno Costa', 'bruno.costa@email.com')
(3, 'Carla Dias', 'carla.dias@email.com')

Produtos com preço > R$200.00:
('Notebook Xpto', 3500.0)
('Teclado Mecânico', 300.0)
('Monitor UltraWide', 1800.0)

Detalhes das Vendas (com Nomes de Cliente e Produto):
ID Venda: 1, Cliente: Ana Silva, Produto: Notebook Xpto, Quantidade: 1, Preço Unitário: 3500.0, Total: 3500.00, Data: 2025-05-12 00:29:36
ID Venda: 2, Cliente: Bruno Costa, Produto: Mouse Gamer, Quantidade: 2, Preço Unitário: 150.0, Total: 300.00, Data: 2025-05-12 00:29:36
ID Venda: 3, Cliente: Ana Silva, Produto: Teclado Mecânico, Quantidade: 1, Preço Unitário: 300.0, Total: 300.00, Data: 20